In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC ### 2. Movimentação: Bronze para Silver
# MAGIC 
# MAGIC **Origem:** Tabela `movies_catalog.bronze.movies_raw`
# MAGIC 
# MAGIC **Destino:** Tabela `movies_catalog.silver.movies_cleaned`
# MAGIC 
# MAGIC **Ações:**
# MAGIC 1. Converte colunas de valores monetários para tipos numéricos.
# MAGIC 2. Converte a coluna de data para o formato `Date` e extrai o ano.
# MAGIC 3. Renomeia colunas para um padrão claro em português.
# MAGIC 4. Remove colunas desnecessárias e seleciona a ordem final.

# COMMAND ----------

from pyspark.sql.functions import col, regexp_replace, to_date, year

# Garante que o catálogo correto está em uso
spark.sql("USE CATALOG movies_catalog")

# COMMAND ----------

# Carrega a tabela da camada Bronze
df_bronze = spark.read.table("bronze.movies_raw")

# Aplica as transformações e limpezas
df_silver = df_bronze \
    .withColumn("orcamento", regexp_replace(col("budget"), "[$,]", "").cast("long")) \
    .withColumn("receita", regexp_replace(col("revenue"), "[$,]", "").cast("long")) \
    .withColumn("data_lancamento", to_date(col("release_date"), "yyyy-MM-dd")) \
    .withColumn("ano_lancamento", year(col("data_lancamento"))) \
    .withColumnRenamed("original_title", "titulo_original") \
    .withColumnRenamed("vote_average", "nota_media") \
    .withColumnRenamed("runtime", "duracao_min") \
    .drop("budget", "revenue", "release_date", "homepage", "tagline")

# Seleciona e reordena as colunas para a tabela final da camada Silver
df_silver_final = df_silver.select(
    "id",
    "titulo_original",
    "data_lancamento",
    "ano_lancamento",
    "genres",
    "orcamento",
    "receita",
    "duracao_min",
    "nota_media",
    "overview"
)

silver_table_name = "silver.movies_cleaned"

# Salva a tabela limpa como uma tabela Delta gerenciada na camada Silver
df_silver_final.write.mode("overwrite") \
               .option("overwriteSchema", "true") \
               .saveAsTable(silver_table_name)

print(f"Tabela '{silver_table_name}' salva com sucesso na camada Silver.")